In [ ]:
# Loading data, library stuff

require 'date'
require 'json'
require 'csv'

def h3 txt
  IRuby.display "<h3>#{txt}</h3>", mime: 'text/html'
end

class Numeric
  def r x
    sprintf("%.#{x}f", self)
  end
end

class Array
  def sum
    inject 0 do |sum,item|
      sum + yield(item)
    end
  end
  def average
    inject(&:+) / size
  end
end

class Range
  def enum_with
    Enumerator.new do |y|
      i = self.begin
      loop do
        y << i
        i = yield i
        break if i > self.end
      end
    end
  end
end

class Date
  def next_week
    self + 7
  end
end

class LongPos
  attr_reader :coin, :price, :amt, :cost, :close_price
  
  def self.buy_for coin, price, cost
    new coin, price, cost/price, cost
  end
  
  def initialize coin, price, amt, cost=price*amt
    @coin, @price, @amt, @cost = coin, price, amt, cost
  end
  
  def close! price
    raise "Cannot close, no price" unless price
    @close_price = price
  end
  
  def close price
    self.class.new(coin, price, amt, cost).tap do |inst|
      inst.close! price
    end
  end
  
  def profit
    close_price*amt - cost if close_price
  end
  
  def gains
    close_price.to_f/price-1 if close_price
  end
  
  def end_value
    close_price*amt
  end
  
  def value_at current_price
    current_price*amt
  end
  
  def render
    { "Coin" => coin, "Price" => price, "Amount" => sprintf("~%.4f", amt),
      "Closed at" => close_price,
      "Cost" => cost.r(2), "Sold for" => end_value.r(2),
      "Profit" => profit.r(2),
      "Gains" => (gains*100).r(2)}
  end
end

class Strategy
  attr_reader :start, :finish, :advance, :initial_funds, :size

  Step = Struct.new(:date, :end_date, :portfolio, :funds, :end_value)

  class << self
    @@tops = {}
    CSV.foreach "tops.csv" do |(ts,*coins)|
      @@tops.store Date.parse(ts), coins
    end
    @@earliest = @@tops.keys.min

    def tops_at date
      date.downto(@@earliest).lazy.map { |d| @@tops[d] }.reject(&:nil?).first
    end
    
    def ohlc open,high,low,close,vol,cap,avg
      [open,high,low,close].average # OHLC average
    end
    
    @@base = 'USD'
    def base; @@base; end

    @@data = Hash.new do |h,coin|
      h[coin] = d = {}
      path = "prices/#{coin.downcase}_#{@@base.downcase}.csv"
      CSV.foreach path do |ts,*data|
        d.store Date.parse(ts), Strategy.ohlc(*data.map(&:to_f))
      end
      d
    end
    
    def price_at coin, date
      @@data[coin][date]
    end
  end
  
  def initialize period, cfg
    @start, @finish = period.begin, period.end
    @tag = cfg[:tag] if cfg[:tag]
    @advance, @initial_funds, @size =
      cfg[:advance].to_proc,
      cfg[:funds], cfg[:size]
  end
  
  def label
    if @tag
      "#{@tag} (#{size})"
    else
      size
    end
  end
  
  def run
    return @res if @res
    @res, funds, period_start = [], initial_funds, start
    while period_start < finish do
      portfolio, stake, period_end = [], funds/@size, advance.(period_start)
      period_end = finish if period_end > finish
      Strategy.tops_at(period_start).each do |coin|
        begin
          item = LongPos.buy_for(coin, Strategy.price_at(coin, period_start), stake)
          item.close! Strategy.price_at(coin, period_end)
          portfolio << item
        rescue
          next
        end
        break if portfolio.size == @size
      end
      endval = portfolio.sum &:end_value
      @res << Step.new(period_start, period_end, portfolio, funds, endval)
      funds, period_start = endval, period_end
    end
    @end_value = funds
    @res
  end
  
  def end_value
    run
    @end_value
  end
end

In [ ]:
### Configuration
# Advance can use one of: next_month (monthly) next_week, (weekly), next (daily)
# Note that top coin data is of weekly granularity.

$epoch = Date.new 2017, 1, 1
$until = Date.new 2018, 1, 1
$span  = $epoch..$until
$funds = 10000

strats = [
  *[10, 20].map { |s| {tag: "Monthly",  advance: :next_month,    size: s} },
  *[10, 20].map { |s| {tag: "Biweekly", advance: ->(d) { d+14 }, size: s} },
  *[10, 20].map { |s| {tag: "Weekly",   advance: :next_week,     size: s} },
].map { |cfg| Strategy.new $span, cfg.merge(funds: $funds) }

# strats = [5, 10, 25, 50].map do |n|
#   Strategy.new $span, tag: "Daily", advance: :next, size: n, funds: $funds
# end

strats.size

In [ ]:
require 'gruff'
Gruff::Line.new(1200).tap do |g|
  g.title = "Value of portfolios over time"
  g.hide_dots = true
  g.line_width = 1
  
  months = $span.enum_with &:next_month  
  g.labels = months.map { |d| [d.to_time.to_i, d.strftime('%b')] }.to_h

  for coin in [] # %w(btc eth)
    coin_amt = $funds/Strategy.price_at(coin, months.first)
    g.dataxy(
      "Holding #{coin.upcase}",
      months.map { |d| [d.to_time.to_i, coin_amt*Strategy.price_at(coin, d)]})
  end
  
  strats.each do |s|
    g.dataxy(
      s.label,
      [[s.start.to_time.to_i, s.initial_funds],
        *s.run.map { |step| [step.end_date.to_time.to_i, step.end_value] }])
  end
end

In [ ]:
strats.first.run.each do |step|
  h3 "#{step.date}, spending #{sprintf('%.2f', step.funds)} #{Strategy.base}"
  IRuby.display IRuby.table(
    [*step.portfolio.map(&:render),
      {"Coin" => "Total", "Cost" => step.funds.r(2), "Sold for" => step.end_value.r(2),
        "Profit" => (step.end_value-step.funds).r(2),
        "Gains" => sprintf("%.2f%", (step.end_value/step.funds-1)*100)}
      ])
end
nil